## Coverting dB Spectogram back to Audio Signal using Phase info, comparing with audio after attack, calculating SNR
- 19/08/2019
- By Karl Michel Koerich

In [1]:
#Imports

import librosa
import librosa.display
import IPython.display
import numpy as np
import math
import os
import soundfile as sf

In [2]:

nb_classes = 10
sr = 22050
frame_size = 110250


In [3]:
# Load 5 second Original STFT Spec

#Indicate folds
train_fold = [2, 3]
test_fold = [1]

X_valid         = np.load( "folds_stft/2_GTzan_Xs_test_fold"   +str(test_fold[0])+"_110250_75_frozen_stft.npy")
#X_phases        = np.load( "folds_stft/2_GTzan_phase_test_fold"+str(test_fold[0])+"_110250_75_frozen_stft.npy")
#X_coeffs        = np.load( "folds_stft/2_GTzan_coefs_test_fold"+str(test_fold[0])+"_110250_75_frozen_stft.npy")
#X_attacked_bim  = np.load( "folds_stft/2_GTzan_Xs_BIM_fold"    +str(test_fold[0])+"_110250_75_frozen_stft.npy")
#X_attacked_fgsm = np.load( "folds_stft/2_GTzan_Xs_FGSM_fold"   +str(test_fold[0])+"_110250_75_frozen_stft.npy")

X_sample = X_valid[9:10]  #Arbitrary values
#X_phase  = X_phases[9:10] #Arbitrary values
#X_coef   = X_coeffs[9:10] #Arbitrary values

In [4]:
# Put all filenames into a list

test_filename_list  = []
base_path = "gtzan_3f_Win_110250_75/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            test_filename_list.append( os.path.join(path, name) )
            j += 1
    print("Test files processed: "+"fold"+str(ii)+" "+str(j) )

Test files processed: fold1 7138


In [5]:
#Load STFT spectrogram wave paths

stft_wave_list  = []
base_path = "gtzan_reconstruct/stft/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_wave_list.append(os.path.join(path, name))
            j += 1
    print("Reconstructed waves processed: "+"fold "+str(ii)+" - "+str(j) )

Reconstructed waves processed: fold 1 - 7138


In [6]:
#Load STFT FGSM attacked wave paths

stft_fgsm_attacked_wave_list  = []
base_path = "gtzan_reconstruct/stft_FGSM/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_fgsm_attacked_wave_list.append(os.path.join(path, name))
            j += 1
    print("Attacked waves processed: "+"fold "+str(ii)+" - "+str(j) )

Attacked waves processed: fold 1 - 7138


In [7]:
#Load STFT BIM attacked wave paths

stft_bim_attacked_wave_list  = []
base_path = "gtzan_reconstruct/stft_BIM/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_bim_attacked_wave_list.append(os.path.join(path, name))
            j += 1
    print("Attacked waves processed: "+"fold "+str(ii)+" - "+str(j) )

Attacked waves processed: fold 1 - 7138


In [8]:
#Load STFT GRIFFIN spectrogram wave paths

stft_griffin_wave_list  = []
base_path = "gtzan_reconstruct/stft_griffin/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_griffin_wave_list.append(os.path.join(path, name))
            j += 1
    print("Reconstructed waves processed: "+"fold "+str(ii)+" - "+str(j) )

Reconstructed waves processed: fold 1 - 7138


In [9]:
#Load STFT GRIFFIN FGSM attacked wave paths

stft_griffin_fgsm_attacked_wave_list  = []
base_path = "gtzan_reconstruct/stft_griffin_FGSM/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_griffin_fgsm_attacked_wave_list.append(os.path.join(path, name))
            j += 1
    print("Attacked waves processed: "+"fold "+str(ii)+" - "+str(j) )

Attacked waves processed: fold 1 - 7138


In [10]:
#Load STFT GRIFFIN BIM attacked wave paths

stft_griffin_bim_attacked_wave_list  = []
base_path = "gtzan_reconstruct/stft_griffin_BIM/"

j  = 0 
ii = 0
for ii in test_fold:

    path = base_path+"fold"+str(ii)

    for path, subdirs, files in os.walk( path ):
        for name in files:
            stft_griffin_bim_attacked_wave_list.append(os.path.join(path, name))
            j += 1
    print("Attacked waves processed: "+"fold "+str(ii)+" - "+str(j) )

Attacked waves processed: fold 1 - 7138


In [11]:

X_sample = np.squeeze(X_sample)
shape_X = X_sample.shape
print(shape_X)
len_X = X_valid.shape[0]
print(len_X)


(257, 431)
7138


In [12]:
#Power function

def power(v):
    n = 0
    s = 0
    for vn in v:
        square = vn*vn
        s += square
        n += 1
    return math.sqrt(s/n)

In [14]:
#Reconstructing original stft and attacked stft specs

snr_original_stft    = np.ones((len_X, 1), dtype=float)
snr_stft_bim         = np.ones((len_X, 1), dtype=float)
snr_stft_fgsm        = np.ones((len_X, 1), dtype=float)
snr_original_griffin = np.ones((len_X, 1), dtype=float)
snr_griffin_bim      = np.ones((len_X, 1), dtype=float)
snr_griffin_fgsm     = np.ones((len_X, 1), dtype=float)

for i in range(0, len_X):
        
    #POWER ORIGINAL
    file_wave, sr = librosa.load(test_filename_list[i])
    power_wave = power(file_wave)
        
    #POWER STFT
    audio_rec_stft, sr = librosa.load(stft_wave_list[i])
    power_stft = power(audio_rec_stft)
    
    #POWER STFT_BIM
    audio_rec_stft_bim, sr = librosa.load(stft_bim_attacked_wave_list[i])
    power_stft_bim = power(audio_rec_stft_bim)
    
    #POWER STFT_FGSM
    audio_rec_stft_fgsm, sr = librosa.load(stft_fgsm_attacked_wave_list[i])
    power_stft_fgsm = power(audio_rec_stft_fgsm)
    
    #POWER NOISE ORIGINAL-STFT
    power_wave_m_stft = abs(power_wave-power_stft)
    if (power_wave_m_stft == 0): power_wave_m_stft = 0.00001
    quo = power_wave/power_wave_m_stft    
    snr = 20*math.log(quo, 10)
    snr_original_stft[i] = snr
    
    #POWER NOISE STFT-BIM
    power_stft_m_bim = abs(power_stft-power_stft_bim)
    if (power_stft_m_bim == 0): power_stft_m_bim = 0.00001
    quo = power_stft/power_stft_m_bim    
    snr = 20*math.log(quo, 10)
    snr_stft_bim[i] = snr
    
    #POWER NOISE STFT-FGSM
    power_stft_m_fgsm = abs(power_stft-power_stft_fgsm)
    if (power_stft_m_fgsm == 0): power_stft_m_fgsm = 0.00001
    quo = power_stft/power_stft_m_fgsm   
    snr = 20*math.log(quo, 10)
    snr_stft_fgsm[i] = snr
    
    #-------------------------------------------------------------#
    
    #POWER STFT_GRIFFIN
    #audio_rec_griffin, sr = librosa.load(stft_griffin_wave_list[i])
    #power_griffin = power(audio_rec_griffin)
    
    #POWER STFT_GRIFFIN_BIM
    #audio_rec_griffin_bim, sr = librosa.load(stft_griffin_bim_attacked_wave_list[i])
    #power_griffin_bim = power(audio_rec_griffin_bim)
    
    #POWER STFT_GRIFFIN_FGSM
    #audio_rec_griffin_fgsm, sr = librosa.load(stft_griffin_fgsm_attacked_wave_list[i])
    #power_griffin_fgsm = power(audio_rec_griffin_fgsm)
    
    #POWER NOISE ORIGINAL-STFT_GRIFFIN
    #power_wave_m_griffin = abs(power_wave-power_griffin)
    #if (power_wave_m_griffin == 0): power_wave_m_griffin = 0.00001
    #quo = power_wave/power_wave_m_griffin    
    #snr = 20*math.log(quo, 10)
    #snr_original_griffin[i] = snr
    
    #POWER NOISE STFT_GRIFFIN-GRIFFIN_BIM
    #power_griffin_m_bim = abs(power_griffin-power_griffin_bim)
    #if (power_griffin_m_bim == 0): power_griffin_m_bim = 0.00001
    #quo = power_griffin/power_griffin_m_bim    
    #snr = 20*math.log(quo, 10)
    #snr_griffin_bim[i] = snr
    
    #POWER NOISE STFT_GRIFFIN-GRIFFIN_FGSM
    #power_griffin_m_fgsm = abs(power_griffin-power_griffin_fgsm)
    #if (power_griffin_m_fgsm == 0): power_griffin_m_fgsm = 0.00001
    #quo = power_griffin/power_griffin_m_fgsm   
    #snr = 20*math.log(quo, 10)
    #snr_griffin_fgsm[i] = snr

    if (i%1 == 0.000):
        print("Files read: " + str(i) )
        
        print("SNR WAVE-STFT:", snr_original_stft[i])
        print("SNR STFT-BIM :", snr_stft_bim[i])
        print("SNR STFT-FGSM:", snr_stft_fgsm[i])
        
        #print("SNR WAVE-GRIFFIN:", snr_original_griffin[i])
        #print("SNR GRIFFIN-BIM :", snr_griffin_bim[i])
        #print("SNR GRIFFIN-FGSM:", snr_griffin_fgsm[i])
        

Files read: 0
SNR WAVE-STFT: [123.05033227]
SNR STFT-BIM : [26.59488242]
SNR STFT-FGSM: [18.16471316]
Files read: 1
SNR WAVE-STFT: [111.42323432]
SNR STFT-BIM : [22.76130835]
SNR STFT-FGSM: [16.17866613]
Files read: 2
SNR WAVE-STFT: [114.90572533]
SNR STFT-BIM : [27.22537627]
SNR STFT-FGSM: [19.60194208]
Files read: 3
SNR WAVE-STFT: [130.67699881]
SNR STFT-BIM : [23.42767148]
SNR STFT-FGSM: [17.81320423]
Files read: 4
SNR WAVE-STFT: [128.65128464]
SNR STFT-BIM : [23.14632553]
SNR STFT-FGSM: [15.85248743]
Files read: 5
SNR WAVE-STFT: [137.92698468]
SNR STFT-BIM : [31.08518577]
SNR STFT-FGSM: [22.25157415]
Files read: 6
SNR WAVE-STFT: [100.63446326]
SNR STFT-BIM : [3.68736023]
SNR STFT-FGSM: [-2.01296747]
Files read: 7
SNR WAVE-STFT: [121.40940933]
SNR STFT-BIM : [25.57508524]
SNR STFT-FGSM: [17.36259295]
Files read: 8
SNR WAVE-STFT: [116.30731535]
SNR STFT-BIM : [20.6014403]
SNR STFT-FGSM: [12.57657877]
Files read: 9
SNR WAVE-STFT: [131.28563961]
SNR STFT-BIM : [28.05590948]
SNR STFT-FG

Files read: 81
SNR WAVE-STFT: [122.9224354]
SNR STFT-BIM : [35.70423236]
SNR STFT-FGSM: [26.00503883]
Files read: 82
SNR WAVE-STFT: [126.05476014]
SNR STFT-BIM : [21.8169974]
SNR STFT-FGSM: [15.0375606]
Files read: 83
SNR WAVE-STFT: [120.35414992]
SNR STFT-BIM : [29.99580231]
SNR STFT-FGSM: [20.43609468]
Files read: 84
SNR WAVE-STFT: [115.46341196]
SNR STFT-BIM : [17.46638943]
SNR STFT-FGSM: [10.96374833]
Files read: 85
SNR WAVE-STFT: [105.70819296]
SNR STFT-BIM : [19.71711644]
SNR STFT-FGSM: [13.64978291]
Files read: 86
SNR WAVE-STFT: [133.19646817]
SNR STFT-BIM : [22.27797451]
SNR STFT-FGSM: [16.41262897]
Files read: 87
SNR WAVE-STFT: [128.91698768]
SNR STFT-BIM : [27.88516954]
SNR STFT-FGSM: [20.82891112]
Files read: 88
SNR WAVE-STFT: [130.36404981]
SNR STFT-BIM : [12.245851]
SNR STFT-FGSM: [6.2614725]
Files read: 89
SNR WAVE-STFT: [120.86879914]
SNR STFT-BIM : [25.76327817]
SNR STFT-FGSM: [18.39524177]
Files read: 90
SNR WAVE-STFT: [116.52157084]
SNR STFT-BIM : [25.2524757]
SNR STF

Files read: 161
SNR WAVE-STFT: [118.69967638]
SNR STFT-BIM : [24.44062817]
SNR STFT-FGSM: [17.8195536]
Files read: 162
SNR WAVE-STFT: [120.06509932]
SNR STFT-BIM : [21.39462534]
SNR STFT-FGSM: [13.95170203]
Files read: 163
SNR WAVE-STFT: [103.87303415]
SNR STFT-BIM : [23.14566052]
SNR STFT-FGSM: [14.66194206]
Files read: 164
SNR WAVE-STFT: [116.47297971]
SNR STFT-BIM : [17.93045011]
SNR STFT-FGSM: [10.9800937]
Files read: 165
SNR WAVE-STFT: [114.39973329]
SNR STFT-BIM : [28.95622472]
SNR STFT-FGSM: [20.10212378]
Files read: 166
SNR WAVE-STFT: [152.45726506]
SNR STFT-BIM : [26.74577746]
SNR STFT-FGSM: [19.68495328]
Files read: 167
SNR WAVE-STFT: [138.69171228]
SNR STFT-BIM : [29.62237535]
SNR STFT-FGSM: [20.40036311]
Files read: 168
SNR WAVE-STFT: [122.61581582]
SNR STFT-BIM : [23.11793732]
SNR STFT-FGSM: [15.23667689]
Files read: 169
SNR WAVE-STFT: [101.90578334]
SNR STFT-BIM : [29.21340331]
SNR STFT-FGSM: [21.02099899]
Files read: 170
SNR WAVE-STFT: [105.4893024]
SNR STFT-BIM : [28.77

Files read: 241
SNR WAVE-STFT: [128.56044481]
SNR STFT-BIM : [23.73276228]
SNR STFT-FGSM: [16.94660664]
Files read: 242
SNR WAVE-STFT: [117.38565894]
SNR STFT-BIM : [22.75879982]
SNR STFT-FGSM: [14.84123253]
Files read: 243
SNR WAVE-STFT: [102.92514031]
SNR STFT-BIM : [19.10829085]
SNR STFT-FGSM: [13.58561934]
Files read: 244
SNR WAVE-STFT: [118.0740971]
SNR STFT-BIM : [25.30954092]
SNR STFT-FGSM: [16.76818845]
Files read: 245
SNR WAVE-STFT: [139.59347311]
SNR STFT-BIM : [28.21640084]
SNR STFT-FGSM: [20.35993234]
Files read: 246
SNR WAVE-STFT: [100.37900237]
SNR STFT-BIM : [19.08122965]
SNR STFT-FGSM: [12.89330732]
Files read: 247
SNR WAVE-STFT: [129.75226897]
SNR STFT-BIM : [25.84663183]
SNR STFT-FGSM: [18.7188225]
Files read: 248
SNR WAVE-STFT: [121.32142787]
SNR STFT-BIM : [27.62930465]
SNR STFT-FGSM: [19.83249096]
Files read: 249
SNR WAVE-STFT: [132.52700167]
SNR STFT-BIM : [32.65443186]
SNR STFT-FGSM: [22.67579741]
Files read: 250
SNR WAVE-STFT: [104.95806707]
SNR STFT-BIM : [23.9

Files read: 322
SNR WAVE-STFT: [120.50506242]
SNR STFT-BIM : [26.20407062]
SNR STFT-FGSM: [19.27069214]
Files read: 323
SNR WAVE-STFT: [127.04104384]
SNR STFT-BIM : [23.45648096]
SNR STFT-FGSM: [16.69563438]
Files read: 324
SNR WAVE-STFT: [120.12173264]
SNR STFT-BIM : [14.42780071]
SNR STFT-FGSM: [8.33359196]
Files read: 325
SNR WAVE-STFT: [128.82510903]
SNR STFT-BIM : [21.39058763]
SNR STFT-FGSM: [13.85163378]
Files read: 326
SNR WAVE-STFT: [108.53579287]
SNR STFT-BIM : [21.866986]
SNR STFT-FGSM: [14.29770875]
Files read: 327
SNR WAVE-STFT: [122.72083239]
SNR STFT-BIM : [19.02184241]
SNR STFT-FGSM: [12.21098447]
Files read: 328
SNR WAVE-STFT: [120.11708637]
SNR STFT-BIM : [22.87525241]
SNR STFT-FGSM: [15.92157869]
Files read: 329
SNR WAVE-STFT: [117.89803579]
SNR STFT-BIM : [19.88017278]
SNR STFT-FGSM: [13.59366948]
Files read: 330
SNR WAVE-STFT: [123.19895008]
SNR STFT-BIM : [23.38754696]
SNR STFT-FGSM: [15.08078471]
Files read: 331
SNR WAVE-STFT: [103.01989995]
SNR STFT-BIM : [22.44

...

Files read: 6784
SNR WAVE-STFT: [107.76338649]
SNR STFT-BIM : [29.40517984]
SNR STFT-FGSM: [22.4344091]
Files read: 6785
SNR WAVE-STFT: [128.98878404]
SNR STFT-BIM : [34.24916954]
SNR STFT-FGSM: [26.72589102]
Files read: 6786
SNR WAVE-STFT: [117.56161213]
SNR STFT-BIM : [19.46717714]
SNR STFT-FGSM: [12.54211442]
Files read: 6787
SNR WAVE-STFT: [105.527459]
SNR STFT-BIM : [26.57919183]
SNR STFT-FGSM: [20.17610189]
Files read: 6788
SNR WAVE-STFT: [103.43867498]
SNR STFT-BIM : [27.89298451]
SNR STFT-FGSM: [21.16070353]
Files read: 6789
SNR WAVE-STFT: [90.7694939]
SNR STFT-BIM : [20.17369155]
SNR STFT-FGSM: [13.69458142]
Files read: 6790
SNR WAVE-STFT: [124.46270028]
SNR STFT-BIM : [24.62113602]
SNR STFT-FGSM: [18.48750278]
Files read: 6791
SNR WAVE-STFT: [103.86805856]
SNR STFT-BIM : [27.23904211]
SNR STFT-FGSM: [20.73179525]
Files read: 6792
SNR WAVE-STFT: [133.66416179]
SNR STFT-BIM : [22.88702107]
SNR STFT-FGSM: [16.67802729]
Files read: 6793
SNR WAVE-STFT: [105.31877871]
SNR STFT-BIM 

Files read: 6864
SNR WAVE-STFT: [165.70919597]
SNR STFT-BIM : [23.12665689]
SNR STFT-FGSM: [16.31827131]
Files read: 6865
SNR WAVE-STFT: [111.05810684]
SNR STFT-BIM : [17.39579846]
SNR STFT-FGSM: [11.65241895]
Files read: 6866
SNR WAVE-STFT: [92.08071929]
SNR STFT-BIM : [10.94439996]
SNR STFT-FGSM: [5.89751961]
Files read: 6867
SNR WAVE-STFT: [116.07471479]
SNR STFT-BIM : [31.39999006]
SNR STFT-FGSM: [24.12421514]
Files read: 6868
SNR WAVE-STFT: [106.27621588]
SNR STFT-BIM : [16.70138108]
SNR STFT-FGSM: [10.40497554]
Files read: 6869
SNR WAVE-STFT: [128.29660526]
SNR STFT-BIM : [23.49928352]
SNR STFT-FGSM: [17.21206794]
Files read: 6870
SNR WAVE-STFT: [130.04771881]
SNR STFT-BIM : [29.79345322]
SNR STFT-FGSM: [22.89625727]
Files read: 6871
SNR WAVE-STFT: [119.24546318]
SNR STFT-BIM : [24.91627683]
SNR STFT-FGSM: [18.98607488]
Files read: 6872
SNR WAVE-STFT: [128.78471067]
SNR STFT-BIM : [27.24365888]
SNR STFT-FGSM: [19.60081365]
Files read: 6873
SNR WAVE-STFT: [104.85611671]
SNR STFT-B

Files read: 6944
SNR WAVE-STFT: [135.34284501]
SNR STFT-BIM : [22.35272393]
SNR STFT-FGSM: [16.19439377]
Files read: 6945
SNR WAVE-STFT: [124.04610489]
SNR STFT-BIM : [23.56897356]
SNR STFT-FGSM: [16.94047351]
Files read: 6946
SNR WAVE-STFT: [94.93294442]
SNR STFT-BIM : [22.84450646]
SNR STFT-FGSM: [15.57225839]
Files read: 6947
SNR WAVE-STFT: [113.36876707]
SNR STFT-BIM : [16.56001926]
SNR STFT-FGSM: [9.90088449]
Files read: 6948
SNR WAVE-STFT: [108.50731598]
SNR STFT-BIM : [14.66707277]
SNR STFT-FGSM: [8.33108577]
Files read: 6949
SNR WAVE-STFT: [120.88585464]
SNR STFT-BIM : [20.01369296]
SNR STFT-FGSM: [12.80485235]
Files read: 6950
SNR WAVE-STFT: [106.86692539]
SNR STFT-BIM : [27.28726913]
SNR STFT-FGSM: [20.20460708]
Files read: 6951
SNR WAVE-STFT: [110.72565264]
SNR STFT-BIM : [10.63956017]
SNR STFT-FGSM: [4.74731336]
Files read: 6952
SNR WAVE-STFT: [105.5826124]
SNR STFT-BIM : [7.83259698]
SNR STFT-FGSM: [2.13154609]
Files read: 6953
SNR WAVE-STFT: [95.40370864]
SNR STFT-BIM : [

Files read: 7023
SNR WAVE-STFT: [131.34575425]
SNR STFT-BIM : [22.94567149]
SNR STFT-FGSM: [16.1455262]
Files read: 7024
SNR WAVE-STFT: [105.58510841]
SNR STFT-BIM : [26.6695682]
SNR STFT-FGSM: [20.29678205]
Files read: 7025
SNR WAVE-STFT: [113.23103318]
SNR STFT-BIM : [30.48382215]
SNR STFT-FGSM: [19.61936963]
Files read: 7026
SNR WAVE-STFT: [144.63093877]
SNR STFT-BIM : [19.35340165]
SNR STFT-FGSM: [13.27312623]
Files read: 7027
SNR WAVE-STFT: [130.13321887]
SNR STFT-BIM : [20.626135]
SNR STFT-FGSM: [13.80173563]
Files read: 7028
SNR WAVE-STFT: [108.62887568]
SNR STFT-BIM : [14.41339899]
SNR STFT-FGSM: [8.01771448]
Files read: 7029
SNR WAVE-STFT: [100.04091748]
SNR STFT-BIM : [26.55029911]
SNR STFT-FGSM: [20.54836615]
Files read: 7030
SNR WAVE-STFT: [145.85332056]
SNR STFT-BIM : [22.13653576]
SNR STFT-FGSM: [15.34231834]
Files read: 7031
SNR WAVE-STFT: [124.37240911]
SNR STFT-BIM : [21.84864592]
SNR STFT-FGSM: [15.58498672]
Files read: 7032
SNR WAVE-STFT: [106.84802547]
SNR STFT-BIM 

Files read: 7102
SNR WAVE-STFT: [123.12060203]
SNR STFT-BIM : [24.4400599]
SNR STFT-FGSM: [17.81080288]
Files read: 7103
SNR WAVE-STFT: [131.86120022]
SNR STFT-BIM : [10.23997958]
SNR STFT-FGSM: [4.33377237]
Files read: 7104
SNR WAVE-STFT: [103.87979103]
SNR STFT-BIM : [23.12402175]
SNR STFT-FGSM: [16.85304196]
Files read: 7105
SNR WAVE-STFT: [132.26602691]
SNR STFT-BIM : [30.64666971]
SNR STFT-FGSM: [22.9340831]
Files read: 7106
SNR WAVE-STFT: [120.43587393]
SNR STFT-BIM : [24.77967031]
SNR STFT-FGSM: [16.63954473]
Files read: 7107
SNR WAVE-STFT: [119.70616022]
SNR STFT-BIM : [17.78035178]
SNR STFT-FGSM: [11.10285893]
Files read: 7108
SNR WAVE-STFT: [158.73565938]
SNR STFT-BIM : [23.96957183]
SNR STFT-FGSM: [16.60319753]
Files read: 7109
SNR WAVE-STFT: [120.31097783]
SNR STFT-BIM : [24.57638903]
SNR STFT-FGSM: [18.35339074]
Files read: 7110
SNR WAVE-STFT: [138.83138518]
SNR STFT-BIM : [31.45492166]
SNR STFT-FGSM: [23.80280336]
Files read: 7111
SNR WAVE-STFT: [128.24942822]
SNR STFT-BI

In [15]:
# SNR WAVE/STFT

print("Fold: " + str(test_fold[0]))
print("Mean SNR: " + str(np.mean(snr_original_stft)))
print("Stnd dev: " + str(np.std(snr_original_stft)))

Fold: 1
Mean SNR: 121.25946191456023
Stnd dev: 13.300217526410046


In [16]:
# SNR STFT/BIM

print("Fold: " + str(test_fold[0]))
print("Mean SNR: " + str(np.mean(snr_stft_bim)))
print("Stnd dev: " + str(np.std(snr_stft_bim)))

Fold: 1
Mean SNR: 20.930284002932158
Stnd dev: 6.1299019920914635


In [17]:
# SNR STFT/FGSM

print("Fold: " + str(test_fold[0]))
print("Mean SNR: " + str(np.mean(snr_stft_fgsm)))
print("Stnd dev: " + str(np.std(snr_stft_fgsm)))

Fold: 1
Mean SNR: 14.42932203657841
Stnd dev: 6.06059103174974


In [18]:
# SNR WAVE/GRIFFIN

#print("Fold: " + str(test_fold[0]))
#print("Mean SNR: " + str(np.mean(snr_original_griffin)))
#print("Stnd dev: " + str(np.std(snr_original_griffin)))

Fold: 1
Mean SNR: 7.369756248847998
Stnd dev: 0.3523269075544385


In [19]:
# SNR GRIFFIN/BIM

#print("Fold: " + str(test_fold[0]))
#print("Mean SNR: " + str(np.mean(snr_griffin_bim)))
#print("Stnd dev: " + str(np.std(snr_griffin_bim)))

Fold: 1
Mean SNR: 9.853466533936524
Stnd dev: 5.162231105553177


In [20]:
# SNR GRIFFIN/FGSM

#print("Fold: " + str(test_fold[0]))
#print("Mean SNR: " + str(np.mean(snr_griffin_fgsm)))
#print("Stnd dev: " + str(np.std(snr_griffin_fgsm)))

Fold: 1
Mean SNR: 4.181790118673153
Stnd dev: 5.017283634064737


In [302]:
i =7109

In [303]:
print(test_filename_list[i])

gtzan_3f_Win_110250_75/fold1/rock/838_20.wav


In [304]:
#Original Sound

print(test_filename_list[i])
sound, sr = sf.read(test_filename_list[i])
IPython.display.Audio(data=sound, rate=sr)

gtzan_3f_Win_110250_75/fold1/rock/838_20.wav


In [305]:
#Reconstructed STFT

print(stft_wave_list[i])
print("SNR: " + str(snr_original_stft[i]))
sound, sr = sf.read(stft_wave_list[i])
IPython.display.Audio(data=sound, rate=sr)

gtzan_reconstruct/stft/fold1/rock/838_20.wav
SNR: [120.31097783]


In [306]:
#Reconstructed BIM

print(stft_bim_attacked_wave_list[i])
print("SNR: " + str(snr_stft_bim[i]))
sound, sr = sf.read(stft_bim_attacked_wave_list[i])
IPython.display.Audio(data=sound, rate=sr)

gtzan_reconstruct/stft_BIM/fold1/rock/838_20.wav
SNR: [24.57638903]


In [307]:
#Reconstructed FGSM

print(stft_fgsm_attacked_wave_list[i])
print("SNR: " + str(snr_stft_fgsm[i]))
sound, sr = sf.read(stft_fgsm_attacked_wave_list[i])
IPython.display.Audio(data=sound, rate=sr)

gtzan_reconstruct/stft_FGSM/fold1/rock/838_20.wav
SNR: [18.35339074]
